In [4]:
import site
from pathlib import Path
site.addsitedir(str(Path.cwd().parents[2]))


from time import time
import numpy as np

import jax.numpy as jnp
from jax import random

from presentation.measurement.measurement import MultiQubitMeasurement
from presentation.measurement.formatting import format_bytes

rng_seed = 42
print(f"Random seed is {rng_seed}")

Random seed is 42


In [5]:
#### PHASE AUGMENTED W STATE

num_qubits = 4

state_dim = 1 << num_qubits                 # bit shifting a ...0001 bitstring is the same as 2**num_qubits

rng_phase = random.PRNGKey(rng_seed)
one_hot_thetas = random.uniform(rng_phase, shape=(num_qubits,), minval=0.0, maxval=2 * jnp.pi)
one_hot_amplitudes = jnp.exp(1j * one_hot_thetas) / jnp.sqrt(num_qubits)

one_hot_indices = 1 << jnp.arange(num_qubits - 1, -1, -1)   # we shift a single bit left with descending ints

w_aug = jnp.zeros(state_dim, dtype=jnp.complex64).at[one_hot_indices].set(one_hot_amplitudes)

print("First 10 amplitudes:")
for idx in range(10):
    print(f"{idx:0{num_qubits}b}: {w_aug[idx]:.8f}")

print(f"\nSize of state vector in memory: {format_bytes(w_aug.nbytes)} \n")

First 10 amplitudes:
0000: 0.00000000+0.00000000j
0001: 0.46211326+0.19092229j
0010: 0.46550879+0.18248716j
0011: 0.00000000+0.00000000j
0100: 0.31050199-0.39190370j
0101: 0.00000000+0.00000000j
0110: 0.00000000+0.00000000j
0111: 0.00000000+0.00000000j
1000: -0.44671717-0.22459693j
1001: 0.00000000+0.00000000j

Size of state vector in memory: 128.00 B 



In [6]:
def save_state_vector_columns(w_aug: jnp.ndarray, file_path: str):
    w_np = jnp.stack([jnp.real(w_aug), jnp.imag(w_aug)], axis=1)
    np.savetxt(file_path, np.array(w_np), fmt="%.10f")


file_name = "w_phase_state.txt"
save_state_vector_columns(w_aug, file_name)

In [7]:
measurement_bases = []

# all Z
measurement_bases.append(['Z'] * num_qubits)

# sliding XX window
for i in range(num_qubits - 1):
    basis_list = ['Z'] * num_qubits
    basis_list[i] = 'X'
    basis_list[i+1] = 'X'
    measurement_bases.append(basis_list)

# sliding XY window
for i in range(num_qubits - 1):
    basis_list = ['Z'] * num_qubits
    basis_list[i] = 'X'
    basis_list[i+1] = 'Y'
    measurement_bases.append(basis_list)


for i, basis in enumerate(measurement_bases):
    print(f"Basis {i:2d}: {''.join(basis)}")

Basis  0: ZZZZ
Basis  1: XXZZ
Basis  2: ZXXZ
Basis  3: ZZXX
Basis  4: XYZZ
Basis  5: ZXYZ
Basis  6: ZZXY


In [8]:
file_name = "w_phase_unique_bases.txt"
with open(file_name, "w") as f:
    for basis in measurement_bases:
        basis_str = " ".join(basis) + " \n"
        f.write(basis_str)

In [9]:
samples_per_basis = 5000
rng_samples = random.PRNGKey(rng_seed)

value_file_name = "w_phase_meas_values.txt"
basis_file_name = "w_phase_meas_bases.txt"

with open(value_file_name, "w") as f_meas, open(basis_file_name, "w") as f_basis:
    for _, measurement_dirs in enumerate(measurement_bases):
        basis_str = " ".join(measurement_dirs) + " \n"
        print(f"Sampling {samples_per_basis} samples for basis {''.join(measurement_dirs)}...")

        measurement = MultiQubitMeasurement(measurement_dirs)

        start = time()
        samples = measurement.sample_state(w_aug, samples_per_basis, rng=rng_samples)
        print(f"  Done in {time() - start:.2f} seconds.")

        for bitstring in samples:
            bits_str = " ".join(map(str, bitstring)) + " \n"
            f_meas.write(bits_str)
            f_basis.write(basis_str)

Sampling 5000 samples for basis ZZZZ...


Constructing basis ZZZZ: 100%|██████████| 16/16 [00:00<00:00, 277.24it/s]


  Done in 0.29 seconds.
Sampling 5000 samples for basis XXZZ...


Constructing basis XXZZ: 100%|██████████| 16/16 [00:00<00:00, 3970.23it/s]


  Done in 0.00 seconds.
Sampling 5000 samples for basis ZXXZ...


...

Constructing basis ZXYZ: 100%|██████████| 16/16 [00:00<00:00, 3646.23it/s]

  Done in 0.00 seconds.


Sampling 5000 samples for basis ZZXY...


Constructing basis ZZXY: 100%|██████████| 16/16 [00:00<00:00, 3010.31it/s]

  Done in 0.00 seconds.
